In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
from common import *

In [2]:
def partition_validation(filename, threshold=0.45, seed=123):
    from collections import defaultdict
    import os

    name = os.path.splitext(os.path.basename(filename))[0]
    out_name = f"./output/{name}.txt"
    command = f"./partition-validation/partition-validation -s {seed} -t {threshold} {filename} {out_name}"
    os.system(command)

    clusters = defaultdict(list)

    with open(out_name) as f:
        next(f), next(f)
        for line in f:
            line = line.strip()
            cluster, partition = map(int, line.split())
            clusters[cluster - 1].append(partition - 1)

    return dict(clusters)

In [3]:
chromosome = 10

In [4]:
import glob

alpha = 127

files = glob.glob(f"matlab/output/A1_chr{chromosome}_gamma*_alpha{alpha}.csv")

def get_gamma(filename):
    return int(filename.split("gamma")[1].split("_", maxsplit=1)[0])

gammas = [get_gamma(f) for f in files]
gammas = set(gammas)
gammas = sorted(gammas)
gammas

[50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120,
 125,
 130,
 135,
 140,
 145,
 150,
 155,
 160,
 165,
 170,
 175,
 180,
 185,
 190,
 195,
 200]

In [6]:
for gamma in gammas:
    threshold = 0.1

    gamma = f"{gamma:.0f}"
    filename = f"A1_chr{chromosome}_gamma{gamma}_alpha{alpha}"

    # all partitions
    run_significance_clustering(f"matlab/output/{filename}.csv", f"output/{filename}.txt")
    write_partition(f"output/{filename}.txt", f"output/{filename}_all.stree")
    
    clusters = partition_validation(f"matlab/output/{filename}_partitions.csv", threshold=threshold)
    continue
    
    #print(clusters)
    
    found = False
    for i, partitions in clusters.items():
        if len(partitions) < 100:
            continue
        #if found:
        #    break
        found = True

        a = np.loadtxt(f"matlab/output/{filename}.csv", delimiter=" ", dtype=np.int32)
        b = a[:, 0]
        for partition in partitions:
            b = np.block([[b], [a[:, partition + 1]]])
        b = b.T

        np.savetxt(f"output/{filename}_cluster{i}.csv", b, delimiter=" ", fmt="%d")

        run_significance_clustering(f"output/{filename}_cluster{i}.csv", f"output/{filename}_{i}.txt")
        
        #modules = pd.read_csv(f"output/{filename}_{i}.txt", comment="#", sep=" ", names=["module", "node"], dtype={"module": str, "node": np.int32})
        #insignificant = modules[modules.module.str.endswith(';')]
        #significant = modules[~modules.module.str.endswith(';')]
        
        #insignificant.module = insignificant.module.str.replace(";", "")
        #insignificant.to_csv(f"output/{filename}_insignificant_{i}.csv", index=False)
        
        #significant.to_csv(f"output/{filename}_significant_{i}.csv", index=False)
        
        write_partition(f"output/{filename}_{i}.txt", f"output/{filename}_{i}.stree")

Multi-level significance clustering
Running with:
	- conf: 0.95
	- seed: 123
	- output: output/A1_chr10_gamma50_alpha127.txt

Reading input file... done (1355 nodes in 33 modules)
Computing similarities... done (18 ms)
Clustering... 33/33 done (1 ms)
Writing output file... done
Version: Nov 5, 2022.
Command: ./partition-validation/partition-validation -s 123 -t 0.1 matlab/output/A1_chr10_gamma50_alpha127_partitions.csv ./output/A1_chr10_gamma50_alpha127_partitions.txt
Setup:
--> Using seed: 123
--> Will cluster partitions such that no partition is farther away from its center than: 0.1
--> Will read partitions from file: matlab/output/A1_chr10_gamma50_alpha127_partitions.csv
--> Will write clusters to file: ./output/A1_chr10_gamma50_alpha127_partitions.txt
--> Will use number of threads: 1
Reading partitions file with 100 partitions...of 1355 nodes...done!
Clustering 100 partitions (modified algorithm):
--> Writing 1 clusters...done!
Multi-level significance clustering
Running with:
	-